<a href="https://colab.research.google.com/github/WaterPurify/NLP_2022/blob/main/Assignment2_Attention_is_All_You_Need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment2- Attention is All You Need


[Attention is All You Need](https://arxiv.org/abs/1706.03762) paper을 바탕으로 notebook에 코드 구현합니다. 모든 이미지들은 Transformer paper로 부터 얻어온 것입니다.

 ![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/transformer1.png?raw=1)

## Introduction

Convolutional Sequence-to-Sequence 모델과 비슷하게 Transformer는 recurrence 사용하지 않습니다. Convolutional layer도 사용하지 않습니다. 모델은 normalization, attention mechanisms과 linear layer로 이루워져 있습니다.
 
 2020년 1월부터 Transformer는 NLP에서 가장 대중적으로 사용한 architecture이고 많은 task에 SOTA 성능을 내주었습니다.

가장 유명한 Transformer variant는 [BERT](https://arxiv.org/abs/1810.04805) (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) 그리고 pre-trained BERT는 NLP에서 embedding layer를 대체하여 많이 적용되었습니다. 

Pre-trained transformer를 사용할 경우 가장 범용적으로 이용되는 library는 [Transformers](https://huggingface.co/transformers/) library입니다. 

Paper와 notebook에서 구현된 것의 차이점들:
- Static encoding 대신에 학습된 positional encoding 적용됨
- 기존 Adam optimizer의 warm-up 와 cool-down steps 대신하여 static learning rate 사용함
- Label smoothing을 사용하지 않음

위의 변화들을 Bert의 set-up과 비슷하게 설정하였습니다.



## Preparing the Data

재활용하기 위해 random seed를 설정하고 필요한 module을 import합니다


In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy

!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_
!pip3 install https://github.com/explosion/spacy-models/releases/download/de_core_n
!pip install torchtext==0.10

import torch
print("torch version:", torch.__version__)
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/en_core_web_
ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/en_core_web_ because of HTTP error 404 Client Error: Not Found for url: https://github.com/explosion/spacy-models/releases/download/en_core_web_ for URL https://github.com/explosion/spacy-models/releases/download/en_core_web_
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/de_core_n
ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/de_core_n because of HTTP error 404 Client Error: Not Found for 

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

그전과 같이 tokenizerfmf 생성합니다


In [4]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


spacy_de = spacy.load('de_core_news_sm')
print(spacy_de)
spacy_en = spacy.load('en_core_web_sm')
print(spacy_en)

2022-11-01 08:00:07.338084: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2022-11-01 08:00:19.297791: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 22.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [5]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]
    
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]


그전에 사용하였던 세팅을 그래도 사용합니다. 모델은 batch 형태로 data를 받는 다고 말하고 있고 'batch_first = True'라고 사용합니다.


In [6]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)


TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

print(TRG)

Multi30k dataset을 load하고 vocabulary를 build 합니다.


In [7]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 469kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 115kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 164kB/s]


In [9]:
#최소 2번 나타나는 vocabulary dictionary를 만들어줌
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

결과적으로 device를 define하고 data iterator를 사용합니다


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
#한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다
  #따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들어야 합니다
  #이를 위해 BucketIterator를 사용합니다
  #배치 크기는 128로 설정합니다


BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device)

# for i, batch in enumerate(train_iterator):
#   src = batch.src
#   trg = batch.trg

  # print(f"첫 번째 배치 크기: " {src.shape})
  #   #현재 배치에 있는 하나의 문장에 포함된 정보 출력
  #   for i in range(src.shape[1]):
  #     print(f"인덱스 {i}: {src[0][i].item()}")

 

## Building the Model

다음으로 모델을 build합니다. 그전 notebook처럼 encoder와 decoder로 이루워져 있습니다. Encoder는 입력과 source sentence (독일어)를 context 벡터로 encoding 하고 decoder는 context vector를 출력/ target sentence (영어)로 decoding합니다
 

### Encoder

ConvSeq2Seq 모델과 비슷하게 Transformer encoder는 모든 source sentence, $X = (x_1, ... ,x_n)$, 를 하나의 context vector, $z$.로 바꾸어 주려고 하지 않습니다. 
그 대신에 context vectors의 sequence $Z = (z_1, ... , z_n)$로 만들어줍니다. 그래서 만약 입력 sequence는 5 token 길이 일때 우리는 $Z = (z_1, z_2, z_3, z_4, z_5)$로 가질 수 있습니다. 왜 우리는 hidden state의 sequence라고 부르지 않고 context vectors의 sequence라고 부를까요? RNN에서 $t$ time의 hidden state는  tokens $x_t$와 그전의 token을 볼 수 있었지만 각 context vector는 입력 sequence 안에서 모든 position의 token을 볼 수 있습니다. Context vector는 한 꺼번에 볼 수 있지만 RNN의 hidden state는 볼 현재와 그전밖에 볼 수 없습니다.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/transformer-encoder.png?raw=1)

첫번째로 token들은 기본적인 embedding layer를 통과합니다. 그런 다음 모델은 recurrent가 없기 때문에 sequence안에서 token의 순서를 알 수 있는 방법이 없다.
positional embedding layer를 통해 해결 할 수 있습니다. ,'<sos>' (start of sequence) token으로, position 0에서 시작하여 입력이 token이 아니고 sequence의 token position을 나타내고 있습니다. 100개의 token 길이까지 받을 수 있는 size 100의 vocabulary의 position embedding입니다. 더 긴 문장들을 handle하고 싶으면 증가시켜도 됩니다.

Original Transformer,Attention is All You Need paper, 구현에서는 positional embedding이 학습이 되지 않습니다. 그 대신 정해진 static embedding을 이용하고 있습니다.
  
다음 token과 positional embedding은 token과 그 position을 sequence안에서 얻을 수 있도록 elementwise로 합쳐준 vector로 만들어 줍니다. 하지만 더해주기 전에 token embedding은 scaling factor,$\sqrt{d_{model}}$, 로 곱해줍니다., where $d_{model}$ 는 hidden dimension 사이즈, `hid_dim`.입니다.
  Scaling factor를 사용하지 않고 모델이 reliable하게 학습하기 힘들고 embedding에 variance를 줄여주는 역활을 합니다. Dropout는 그런 다음 combined embedding할 경우 사용하였습니다.

합펴진 embedding은 $N$ *encoder layers*를 통해 $Z$, 출력과 decoder로 사용될 수 있는,를 얻을 수 있습니다.

Source mask,`src_mask`,는 source sentence와 같은 shape이고 source sentence에서 token이 `<pad>` token이 아닐때 1의 값을 가지고 있고 `<pad>`일때 0을 출력합니다.
Source sentence에서 attention을 적용하고 계산에 사용되기 위해서 multi-head attention mechanism을 mask하기 위해 encoder단에서 사용됩니다. 모델은 `<pad>` token들에 주목을 하지 않게 하기 위함입니다.


In [17]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        #Token embedding와 position embedding 정의
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        #Dropuout
        self.dropout = nn.Dropout(dropout)
        #Scale= sqrt(hidden_dim)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        print(batch_size)
        src_len = src.shape[1]
        print(src_len)
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        print(pos)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        print(src)
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

### Encoder Layer

Encoder layer는 encoder의 핵심이 들어 있습니다. 첫번째로 source sentence로 통과 시키고 mask를 multi-head attention layer롤 통과 시킵니다. 그리고 dropout, residucal connection과 Layer Normalization layer를 통과시킵니다. 그리고 pointwise feedforward를 통과 시키고 다시 한번 dropout, residual connection과 layer normalization를 얻을 수 있고 다음 layer로 입력됩니다. Parameter는 각 layer마다 공유되지 않습니다.

Source sentence에 다시 연결되는 encoder layer를 multi-head attention layer가 사용되고 있습니다.

Layer Normalization을 사용하는 이유는 feature를 0을 mean으로 하고 standard deviation을 1로 하는 feature로 만들기 위해서 사용됩니다. 많은 layer를 갖고 있는 neural network 특성상 학습을 빠르게 하기위해 사용됩니다.


In [18]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        #hidden_dim: 하나의 단어의 대한 임베팅 차원
        #n_heads: 헤드의 개수 =scaled dot-product attention의 개수
        #pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
        #dropout_ratio: 드롭아웃 비율
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

### Mutli Head Attention Layer

Multi-head attention layer는 Transformer paper에서 중요한 새로운 concept입니다.


![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/transformer-attention.png?raw=1)

Attention은 queries, keys, values로 이루워져 있습니다. query는 key value와 함께 attention vector를 만드는데 이용됩니다.
 <!-- (usually the output of a *softmax* operation and has all values between 0 and 1 which sum to 1) -->
그런 다음에 value의 weighted sum의 값을 가질 수 있습니다.

Transformer는 scaled dot-product attention을 사용합니다. query와 key가 dot product를 하고 softmax를 적용하고  $d_k$로 scale로 나눠주고 Value로 곱해줍니다.
$d_k$ 는 *head dimension*, `head_dim`를 나타내고 있습니다.

$$ \text{Attention}(Q, K, V) = \text{Softmax} \big( \frac{QK^T}{\sqrt{d_k}} \big)V $$ 

Dot product attention과 비슷하지만 $d_k$로 scale을 주었다는 것이 다른 점입니다.
위 논문에서 dot product의 출력값이 더 이상 커지지 않도록 하는 역활을 하고 gradient를 작게 만들어 주기도 합니다. 

"We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients"

Queries, keys과 values를 단순하게 scaled dot-product attention에 적용되지 않았습니다. Attention application 마다 하나의 concept에 집중하는 것보다 $h$에 집중을 하였습니다. 그리고 `hid_dim` shape으로 head를 다시 결합하였습니다. `hid_dim`가 각자 다른 $h$ concept에 attention할 수 있게 하기 위함입니다.

$$ \text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1,...,\text{head}_h)W^O $$

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V) $$

$W^O$ 가 가장 마지막 multi-head attention layer,`fc`,에 적용된 linear layer입니다. $W^Q, W^K, W^V$ 는 the linear layers `fc_q`, `fc_k` and `fc_v`입니다.

모듈을 통해 먼저 선형 레이어인 'fc_q', 'fc_k', 'fc_v'를 사용하여 $QW^Q$, $KW^K$ 및 $VW^V$를 계산하여 'Q', 'K', 'V'를 제공합니다. 다음으로 query, key, value의 'hid_dim'을 '.view'를 사용하여 'n_heads'로 분할하고 이들을 올바르게 순열하여 함께 곱할 수 있도록 합니다. 그런 다음 'Q'와 'K'를 함께 곱하고 'hid_dim/n_heads'로 계산되는 'head_dim'의 제곱근으로 스케일링하여 '에너지'(비정규화된 주의)를 계산합니다. 그런 다음 에너지를 마스킹하여 시퀀스의 어떤 요소에도 주의를 기울이지 않도록 한 다음 소프트맥스를 적용하고 드롭아웃합니다. 그런 다음 'n_heads'를 함께 결합하기 전에 값 헤드인 'V'에 attention을 적용합니다. 마지막으로, 우리는 'fc_o'로 표현되는 이 $W^O$를 곱한다.

우리의 구현에서 key와 value의 길이는 항상 같으므로, 행렬이 소프트맥스의 출력인 'attention'와 'V'를 곱할 때 우리는 항상 행렬 곱셈을 위한 유효한 치수 크기를 가질 것이다. 이 곱셈은 두 텐서가 모두 2차원일 때 각 텐서의 마지막 2차원에 걸쳐 일괄 행렬 곱셈을 수행하는 'torch.matmul'을 사용하여 수행된다. 이 값은 배치 크기 및 각 헤드에 대해 **[query len, key len] x [value len, head dim]** 배치 행렬 곱셈으로, **[batch size, n heads, query len, head dim]* 결과를 제공합니다.

처음에 이상하게 보이는 한 가지는 dropout이 바로 attention에 적용된다는 것입니다. 이것은 우리의 attention 벡터가 아마도 1로 통합되지 않을 것이고 우리는 토큰에 완전히 주의를 기울일 수 있지만 그 토큰에 대한 주의는 dropout에 의해 0으로 설정된다는 것을 의미한다. 이는 결코 설명되지 않거나 논문에서 언급되지 않았지만, [공식 구현](https://github.com/tensorflow/tensor2tensor/))과 [BERT 포함](https://github.com/google-research/bert/)) 이후 모든 Transformer 구현에서 사용됩니다.
 


In [19]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim #임베팅 차원
        self.n_heads = n_heads #헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hid_dim // n_heads #각 헤드(head)에서의 임베딩 차원
        # Query, key, value 
        self.fc_q = nn.Linear(hid_dim, hid_dim)# Query 값에 적용될 FC layer
        self.fc_k = nn.Linear(hid_dim, hid_dim)# Key 값에 적용될 FC layer
        self.fc_v = nn.Linear(hid_dim, hid_dim)# Value 값에 적용될 FC layer
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        

        print(Q)
        print(K)
        print(V)
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
        #view 함수를 통해 shape을 바꿔줌
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        

        print(Q)
        print(K)
        print(V)
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
        #Q와 K의 matmul하고 scale로 나눠줌
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        print(energy.shape)
        #energy = [batch size, n heads, query len, key len]
        #Mask가 있을 경우 mask를 채워줌
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        #Softmax함수 적용
        attention = torch.softmax(energy, dim = -1)
        print(attention.shape)  
        #attention = [batch size, n heads, query len, key len]
        #V와 matmul을 적용      
        x = torch.matmul(self.dropout(attention), V)
        

        print(x.shape)
        #x = [batch size, n heads, query len, head dim]
        #새로운 메모리를 할당하기 위해 사용
        x = x.permute(0, 2, 1, 3).contiguous()
        
        print(x.shape)
        #x = [batch size, query len, n heads, head dim]
        #
        x = x.view(batch_size, -1, self.hid_dim)
        
        print(x)
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

### Position-wise Feedforward Layer

인코더 레이어 내부의 다른 주요 블록은 *position-wise feedforward layer* 이것은 다중 헤드 주의 레이어에 비해 상대적으로 간단합니다. 입력은 'hid_dim'에서 'pf_dim'으로 변환되며, 여기서 'pf_dim'은 보통 'hid_dim'보다 훨씬 큽니다. 원래의 트랜스포머는 512의 hid_dim과 2048의 pf_dim을 사용하였습니다. ReLU 활성화 함수와 dropout은 다시 'hid_dim' 표현으로 변환되기 전에 적용됩니다.



In [20]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

### Decoder

디코더의 목표는 소스 문장 ,$Z$, 인코딩된 표현을 가져오고 문장, $\hat{Y}$, 타겟 문장에서 token을 예측할 수 있게 바꾸어 줍니다. 그런 다음 $\hat{Y}$은 목표 문장 $Y$에 있는 실제 토큰을 사용하여 손실을 계산하며, 이는 매개 변수의 기울기를 계산한 다음 예측을 개선하기 위해 가중치를 업데이트하는 데 사용됩니다.



![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/transformer-decoder.png?raw=1)

디코더는 인코더와 유사하지만, 현재 두 개의 다중 헤드 attention layers를 가지고 있습니다. Target sequence에 *마스크된 다중 head attention layer*과 디코더 표현을 Query로, 인코더 표현을 Key와 Value로 사용하는 다중 헤드 주의 계층.

디코더는 위치 임베딩을 사용하고 요소별 합을 통해 이들을 스케일링된 임베디드 대상 토큰과 결합하고 이어서 드롭아웃을 한다. 다시 말하지만, 우리의 위치 인코딩은 100의 "어휘"를 가지고 있는데, 이것은 그들이 최대 100개의 토큰을 시퀀스를 받아들일 수 있다는 것을 의미한다. 원하는 경우 이 값을 늘릴 수 있습니다.

그런 다음 결합된 임베딩은 인코딩된 소스인 'enc_src', 소스 및 대상 마스크와 함께 $N$ 디코더 레이어를 통과한다. 인코더의 레이어 수가 모두 $N$로 표시되더라도 디코더의 레이어 수와 같을 필요는 없다.

$N^{th}$ 레이어 이후의 디코더 표현은 선형 레이어인 'fc_out'을 통과한다. PyTorch에서 소프트맥스 연산은 손실 함수 내에 포함되므로 여기서 소프트맥스 계층을 명시적으로 사용할 필요가 없다.

모델이 '<pad>' 토큰에 참석하는 것을 방지하기 위해 인코더에서 했던 것처럼 소스 마스크를 사용할 뿐만 아니라 대상 마스크도 사용한다. 이는 인코더와 디코더를 모두 캡슐화하는 'Seq2Seq' 모델에서 더 자세히 설명하겠지만, 그 요지는 컨볼루션 시퀀스 투 시퀀스 모델에서 디코더 패딩과 유사한 작업을 수행한다는 것이다. 모든 대상 토큰을 동시에 처리하기 때문에 대상 시퀀스의 다음 토큰이 무엇인지 단순히 "보고" 출력함으로써 디코더가 "부정"하는 것을 막는 방법이 필요하다.

우리의 디코더 계층은 또한 정규화된 주의 값을 출력하므로 나중에 모델을 실제로 주목하는 것을 볼 수 있도록 플롯할 수 있다.

In [21]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            #Source mask와 target mask 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

### Decoder Layer

앞서 언급했듯이, 디코더 계층은 현재 'self_attention'과 'encoder_attention'이라는 두 개의 다중 헤드 주의 계층을 가지고 있다는 점을 제외하고는 인코더 계층과 유사합니다.

첫 번째는 인코더에서와 마찬가지로 Query, Key , Value 과 같은 디코더 표현을 사용하여 자체 주의를 수행한다. 여기에는 dropout, residual connection , layer normalization가 뒤따른다. 이 'self_attention' 계층은 대상 시퀀스 마스크인 'trg_mask'를 사용하여 대상 문장의 모든 토큰을 병렬로 처리할 때 디코더가 현재 처리 중인 토큰보다 "앞" 있는 토큰에 attention을 기울임으로써 디코더가 "부정"하는 것을 방지한다.

두 번째는 인코딩된 소스 문장 'enc_src'를 디코더에 실제로 공급하는 방법입니다. 이 다중 헤드 주의 계층에서 Query는 디코더 표현이고 Key와 Values는 인코더 표현이다. 여기서 소스 마스크인 'src_mask'는 다중 헤드 주의 계층이 소스 문장 내의 'pad' 토큰에 참여하는 것을 방지하기 위해 사용됩니다. 그런 다음 dropout, residual connection,layer normalization이 뒤따른다.

마지막으로, 우리는 이것을 위치별 feedforward layer와 또 다른 dropout, residual connection, layer normalization의 시퀀스를 통과합니다.

디코더 계층은 새로운 개념을 도입하지 않고, 단지 인코더와 같은 계층 세트를 약간 다른 방식으로 사용합니다.




In [22]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

### Seq2Seq

마지막으로, 우리는 인코더와 디코더를 캡슐화하고 마스크 생성을 처리하는 'Seq2Seq' 모듈을 가지고 있다.

소스 마스크는 소스 시퀀스가 "<pad>" 토큰과 같지 않은 위치를 확인하여 생성됩니다. 토큰이 "<pad>" 토큰이 아닌 경우 1이고 토큰일 경우 0입니다. 그런 다음 **_[배치 크기, n 헤드, 시퀀스, 시퀀스]_** 형태의 '에너지'에 마스크를 적용할 때 올바르게 방송될 수 있도록 압축 해제된다.

대상 마스크가 약간 더 복잡합니다. 먼저 소스 마스크와 마찬가지로 '<pad>' 토큰에 대한 마스크를 만듭니다. 다음으로, 우리는 'torch.tril'을 사용하여 '후속' 마스크인 'trg_sub_mask'를 생성한다. 이렇게 하면 대각선 위의 요소가 0이 되고 대각선 아래의 요소가 입력 텐서가 무엇이든 간에 설정되는 대각선 행렬이 생성됩니다. 이 경우, 입력 텐서는 1로 채워진 텐서가 될 것이다. 따라서 이것은 우리의 'trg_sub_mask'가 (5개의 토큰을 가진 대상의 경우) 다음과 같이 보인다는 것을 의미한다.

$$\begin{matrix}
1 & 0 & 0 & 0 & 0\\
1 & 1 & 0 & 0 & 0\\
1 & 1 & 1 & 0 & 0\\
1 & 1 & 1 & 1 & 0\\
1 & 1 & 1 & 1 & 1\\
\end{matrix}$$

각 대상 토큰(행)이 볼 수 있는 항목(열)을 표시합니다. 첫 번째 대상 토큰의 마스크는 **_[1, 0, 0, 0]_*이므로 첫 번째 대상 토큰만 볼 수 있습니다. 두 번째 대상 토큰은 **_[1, 1, 0, 0, 0]_*의 마스크를 가지고 있으며, 이는 첫 번째 대상 토큰과 두 번째 대상 토큰을 모두 볼 수 있음을 의미합니다.

그런 다음 "후속" 마스크는 논리적으로 패딩 마스크와 함께 처리되며, 이는 후속 토큰과 패딩 토큰을 모두 처리할 수 없도록 두 개의 마스크를 결합한다. 예를 들어, 마지막 두 토큰이 '<pad>' 토큰인 경우 마스크는 다음과 같습니다.

$$\begin{matrix}
1 & 0 & 0 & 0 & 0\\
1 & 1 & 0 & 0 & 0\\
1 & 1 & 1 & 0 & 0\\
1 & 1 & 1 & 0 & 0\\
1 & 1 & 1 & 0 & 0\\
\end{matrix}$$

마스크가 생성된 후, 그들은 소스 및 대상 문장과 함께 인코더 및 디코더를 사용하여 소스 시퀀스에 대한 디코더의 주의와 함께 예측 대상 문장인 '출력'을 얻는다.

In [23]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

## Training the Seq2Seq Model

이제 인코더와 디코더와 디코더와 디코더를 정의할 수 있습니다.이 모델은 오늘날 연구에서 사용되는 변환기보다 훨씬 작지만 단일 GPU를 빠르게 실행할 수 있습니다.

In [24]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

그런 다음 이를 사용하여 전체 시퀀스 간 캡슐화 모델을 정의합니다. 


In [25]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

우리는 컨볼루션 시퀀스에 대한 37M보다 훨씬 적은 수의 매개 변수를 확인할 수 있습니다.

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,038,341 trainable parameters


이 논문은 어떤 무게 초기화 방식이 사용되었는지 언급하지 않지만, 자비에 유니폼은 트랜스포머 모델들 사이에서 흔한 것 같아서 우리는 여기서 그것을 사용한다.

In [27]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [28]:
model.apply(initialize_weights);

원래 변환기 논문에서 사용되는 최적화는 "warm-up"을 사용하는 학습률을 가진 학습율을 사용하는 학습율을 사용한다.BERT 및 기타 변환기 모델은 ADAM을 사용하여 ADAM을 사용할 것입니다.[다음](http://nlp.seas.harvard.edu/2018/04/03/attention.html#optimizer)]( link 링크를 참조하십시오.

ADAM 또는 다른 학습률은 ADAM 또는 다른 학습 속도가 불안정합니다.

In [29]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

다음으로, 우리는 손실 함수를 정의하여 '<pad>' 토큰을 통해 계산된 손실을 무시하도록 한다.

In [30]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

그런 다음, 우리의 훈련 루프를 정의하겠습니다. 이것은 이전 튜토리얼에서 사용한 것과 정확히 동일합니다.

우리는 모델이 '<eos>' 토큰을 예측하기를 원하지만 모델에 입력이 되지 않기 때문에 단순히 시퀀스 끝에서 '<eos>' 토큰을 잘라낸다. 따라서:

$$\begin{align*}
\text{trg} &= [sos, x_1, x_2, x_3, eos]\\
\text{trg[:-1]} &= [sos, x_1, x_2, x_3]
\end{align*}$$

$x_i$는 실제 목표 시퀀스를 나타냅니다.그러면 우리는 이 모델을 예측해야 할 예측 시퀀스를 예측해야 한다는 예측 시퀀스를 얻을 수 있습니다.

$$\begin{align*}
\text{output} &= [y_1, y_2, y_3, eos]
\end{align*}$$

$y_i$는 예측된 대상 시퀀스 요소를 나타낸다. 그런 다음 우리는 정면을 잘라낸 'sos' 토큰과 함께 원래의 'trg' 텐서를 사용하여 손실을 계산하고, 'eos' 토큰을 남긴다.

$$\begin{align*}
\text{output} &= [y_1, y_2, y_3, eos]\\
\text{trg[1:]} &= [x_1, x_2, x_3, eos]
\end{align*}$$

그런 다음 손실을 계산하고 매개 변수를 표준으로 업데이트합니다.

In [31]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

평가 루프는 그레이디언트 계산 및 매개 변수 업데이트 없이 훈련 루프와 동일하다.

In [32]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

그런 다음 우리는 한 epoch이 얼마나 걸리는지 알려주는 데 사용할 수 있는 작은 함수를 정의합니다.

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

마지막으로 진짜 모델을 학습합니다. 이 모델은 convolutional sequence-to-sequence 모델보다 3배 정도 빠르고 validation 복잡도가 낮습니다.


In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Streaming output truncated to the last 5000 lines.
           -2.0968e+00, -1.5207e+00],
          [ 7.2403e-01, -4.6619e-01,  6.6356e-02,  ...,  2.0454e+00,
           -6.3342e-01, -3.0416e+00]],

         [[ 1.0992e+00,  6.3524e-01, -3.9973e+00,  ..., -1.1776e+00,
           -1.2297e-01, -9.8568e-01],
          [ 1.3393e+00,  2.4647e+00, -2.2607e+00,  ..., -1.2377e+00,
            1.7718e+00,  7.3490e-01],
          [-1.3832e-01, -1.4589e+00, -1.5725e+00,  ..., -6.6041e-01,
            7.4672e-01,  1.2303e+00],
          ...,
          [-5.6152e-01,  1.5082e-01,  2.4897e-01,  ...,  1.1567e+00,
           -2.6699e+00,  1.8510e-01],
          [-6.3305e-01,  1.6047e-01, -5.4145e-01,  ...,  2.2799e-01,
           -1.8788e+00, -9.9967e-01],
          [-9.7416e-01, -3.2556e-01, -6.9693e-01,  ...,  5.7968e-01,
           -2.6161e+00, -7.8504e-01]],

         [[ 4.2755e-01,  2.0008e+00,  4.7595e-01,  ..., -1.7867e+00,
           -1.6914e+00,  9.5578e-01],
          [-4.0162e-01, -7.3129e-01,

가장 좋은 parameter를 load하고 그전 모델과 비교했을때 성능이 더 좋은 test 복잡도를 얻을 수 있습니다


In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

## Inference


이제 우리는 아래의 'translate_sentence' 기능으로 우리 모델에서 번역을 할 수 있다.

수행한 단계는 다음과 같습니다.
- 원본 문장을 토큰화하지 않은 경우 토큰화(문자열)
- "<br>"와 "br>" 토큰을 추가하다
- 원문을 수치화하다
- 그것을 텐서로 변환하고 배치 치수를 추가한다.
- 원본 문장 마스크를 작성하다
- 원본 문장과 마스크를 인코더에 입력한다.
- '<br>' 토큰으로 초기화된 출력 문장을 저장할 목록을 만듭니다.
- 우리가 최대한의 길이에 도달하지 않은 동안.
- 현재 출력 문장 예측을 배치 차원을 가진 텐서로 변환합니다.
- 대상 문장 마스크를 작성하다
- 현재 출력, 인코더 출력 및 두 마스크를 모두 디코더에 넣습니다.
- 주의와 함께 디코더에서 다음 출력 토큰 예측을 가져옵니다.
- 현재 출력 문장 예측에 예측 추가
- 예측이 '<br>' 토큰이었다면 깨졌다.
- 출력 문장을 색인에서 토큰으로 변환하다
- 출력 문장('<br>' 토큰이 제거된 상태)과 마지막 레이어에서 주의를 반환한다.

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

이제 디코딩의 소스 문장을 표시하는 기능을 정의할 수 있습니다.이 모델은 8개의 heads에 attention이 사용되었습니다

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

Training set으로 부터 예시를 가져올 수 있습니다.


In [35]:
example_idx = 8

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['eine', 'frau', 'mit', 'einer', 'großen', 'geldbörse', 'geht', 'an', 'einem', 'tor', 'vorbei', '.']
trg = ['a', 'woman', 'with', 'a', 'large', 'purse', 'is', 'walking', 'by', 'a', 'gate', '.']


번역이 성공적으로 되었지만 우리의 모델은 is walking by를 walks by로 바꾸어줍니다. 의미는 똑같습니다.


In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')



우리는 아래 각 head의 attention을 볼 수 있다. 각각은 확실히 다르지만, head가 실제로 무엇을 학습한 것인지에 대해 추론하는 것은 어렵다(아마도 불가능할 것이다). 어떤 heads은 a를 번역할 때 "eine"에 완전히 주의를 기울이고, 어떤 머리들은 전혀 그렇지 않고, 어떤 머리들은 조금 한다. 그들은 모두 유사한 "하향 계단" 패턴을 따르는 것으로 보이며, 마지막 두 토큰을 출력할 때의 주의는 입력 문장의 마지막 두 토큰에 동일하게 퍼져 있다.

In [ ]:
from matplotlib import pyplot as plt
display_attention(src, translation, attention)

그런다음 validation set으로 부터 
Next, let's get an example the model has not been trained on from the validation set.

In [40]:
example_idx = 6

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['ein', 'brauner', 'hund', 'rennt', 'dem', 'schwarzen', 'hund', 'hinterher', '.']
trg = ['a', 'brown', 'dog', 'is', 'running', 'after', 'the', 'black', 'dog', '.']



모델은 *is running*을 *runs*으로 전환하여 변환하지만, 이는 허용 가능한 스왑입니다.

In [41]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

Streaming output truncated to the last 5000 lines.
          [-1.1917,  2.3701,  2.0445,  ..., -1.5697, -1.4578, -1.1221],
          [-1.0882,  0.4858,  1.0738,  ..., -1.2963, -3.3216,  0.2304]],

         ...,

         [[ 0.1442,  0.5932,  1.8570,  ...,  0.4379, -1.3178,  0.1875],
          [ 0.6411,  0.9577,  0.9245,  ...,  0.0476, -1.3960, -1.4329],
          [-1.8938,  0.1622,  0.8985,  ...,  1.7658, -0.0230, -1.8423],
          ...,
          [-1.1553, -0.0288, -1.4428,  ...,  1.5120, -2.0123, -0.8382],
          [ 0.6153, -0.8370,  0.2847,  ...,  1.1211,  0.4545, -0.9316],
          [ 0.2783,  0.7288,  0.0426,  ..., -0.7510, -0.3007, -1.8194]],

         [[ 0.4533,  0.0563, -0.1383,  ...,  0.2364, -1.1408,  0.2900],
          [-0.6006, -0.2335,  1.0526,  ...,  1.1292, -0.3142,  0.0245],
          [-0.1656,  0.2013, -1.5837,  ...,  0.5364,  1.5156,  0.3064],
          ...,
          [-1.1179, -0.0850,  0.6707,  ..., -0.3032, -0.1270, -1.2342],
          [-1.1678, -0.0298, -1.1849


다시, 어떤 사람들은 "ein"에 완전히 주의를 기울이는 반면, 어떤 사람들은 그것에 주의를 기울이지 않는다. 다시 말하지만, 대부분의 머리들은 마침표를 출력할 때 원본 문장의 <eos> 토큰과 예측 대상 문장의 <eos> 문장에 모두 주의를 분산시키는 것처럼 보이지만, 일부는 문장 시작 부근에서 토큰에 주의를 기울이는 것처럼 보인다.

In [42]:
display_attention(src, translation, attention)

NameError: ignored


Finally, we'll look at an example from the test data.

In [ ]:
example_idx = 10

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.']
trg = ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']


A perfect translation!

In [43]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

Streaming output truncated to the last 5000 lines.
          [-1.1917,  2.3701,  2.0445,  ..., -1.5697, -1.4578, -1.1221],
          [-1.0882,  0.4858,  1.0738,  ..., -1.2963, -3.3216,  0.2304]],

         ...,

         [[ 0.1442,  0.5932,  1.8570,  ...,  0.4379, -1.3178,  0.1875],
          [ 0.6411,  0.9577,  0.9245,  ...,  0.0476, -1.3960, -1.4329],
          [-1.8938,  0.1622,  0.8985,  ...,  1.7658, -0.0230, -1.8423],
          ...,
          [-1.1553, -0.0288, -1.4428,  ...,  1.5120, -2.0123, -0.8382],
          [ 0.6153, -0.8370,  0.2847,  ...,  1.1211,  0.4545, -0.9316],
          [ 0.2783,  0.7288,  0.0426,  ..., -0.7510, -0.3007, -1.8194]],

         [[ 0.4533,  0.0563, -0.1383,  ...,  0.2364, -1.1408,  0.2900],
          [-0.6006, -0.2335,  1.0526,  ...,  1.1292, -0.3142,  0.0245],
          [-0.1656,  0.2013, -1.5837,  ...,  0.5364,  1.5156,  0.3064],
          ...,
          [-1.1179, -0.0850,  0.6707,  ..., -0.3032, -0.1270, -1.2342],
          [-1.1678, -0.0298, -1.1849

In [44]:
display_attention(src, translation, attention)

NameError: ignored

## BLEU

마지막으로 우리는 Transformer에 대한 BLEU 점수를 계산합니다



In [46]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

우리는 Convolutional sequence-to-sequence 모델의 ~34와 주의 기반 RNN 모델의 ~28을 능가하는 36.52의 BLEU 점수를 받는다. 이 모든 것이 최소한의 매개 변수와 가장 빠른 교육 시간을 가지면서 이루어집니다!

In [47]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

Streaming output truncated to the last 5000 lines.

         [[ 7.9241e-02,  2.8128e-02, -4.7145e-01,  ...,  3.6928e-01,
            4.5070e-01, -5.5528e-01],
          [ 1.9696e-01,  9.7789e-01,  4.3741e-01,  ..., -4.8241e-01,
           -8.1256e-01,  1.9263e-01],
          [-9.9352e-01,  4.4133e-01,  8.4768e-01,  ...,  5.7307e-04,
           -1.1207e-01,  8.5537e-01],
          ...,
          [ 3.7138e-01,  4.0717e-01,  3.8016e-01,  ..., -6.1316e-01,
           -5.5394e-01,  2.7325e-01],
          [-8.7367e-02,  2.6417e-01,  7.5434e-02,  ...,  8.1472e-03,
            1.4291e-01, -4.9599e-01],
          [ 7.3648e-01, -5.7454e-01,  6.8018e-01,  ..., -5.7492e-01,
            5.7032e-01,  7.4837e-01]],

         [[ 6.2890e-03,  2.9670e-01,  5.6971e-01,  ...,  1.1698e-01,
           -3.5346e-02, -2.5102e-01],
          [ 1.5884e-01,  5.5968e-01, -8.7075e-01,  ...,  7.0135e-01,
           -7.1382e-01,  3.4067e-01],
          [-8.2308e-01,  1.4854e-01, -1.6427e-01,  ...,  1.1447e-01,
      

Congratulations for finishing these tutorials! I hope you've found them useful.

If you find any mistakes or want to ask any questions about any of the code or explanations used, feel free to submit a GitHub issue and I will try to correct it ASAP.

## Appendix

The `calculate_bleu` function above is unoptimized. Below is a significantly faster, vectorized version of it that should be used if needed. Credit for the implementation goes to [@azadyasar](https://github.com/azadyasar).

In [48]:
def translate_sentence_vectorized(src_tensor, src_field, trg_field, model, device, max_len=50):
    assert isinstance(src_tensor, torch.Tensor)

    model.eval()
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
    # enc_src = [batch_sz, src_len, hid_dim]

    trg_indexes = [[trg_field.vocab.stoi[trg_field.init_token]] for _ in range(len(src_tensor))]
    # Even though some examples might have been completed by producing a <eos> token
    # we still need to feed them through the model because other are not yet finished
    # and all examples act as a batch. Once every single sentence prediction encounters
    # <eos> token, then we can stop predicting.
    translations_done = [0] * len(src_tensor)
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        pred_tokens = output.argmax(2)[:,-1]
        for i, pred_token_i in enumerate(pred_tokens):
            trg_indexes[i].append(pred_token_i)
            if pred_token_i == trg_field.vocab.stoi[trg_field.eos_token]:
                translations_done[i] = 1
        if all(translations_done):
            break

    # Iterate through each predicted example one by one;
    # Cut-off the portion including the after the <eos> token
    pred_sentences = []
    for trg_sentence in trg_indexes:
        pred_sentence = []
        for i in range(1, len(trg_sentence)):
            if trg_sentence[i] == trg_field.vocab.stoi[trg_field.eos_token]:
                break
            pred_sentence.append(trg_field.vocab.itos[trg_sentence[i]])
        pred_sentences.append(pred_sentence)

    return pred_sentences, attention

In [49]:
from torchtext.data.metrics import bleu_score

def calculate_bleu_alt(iterator, src_field, trg_field, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg
            _trgs = []
            for sentence in trg:
                tmp = []
                # Start from the first token which skips the <start> token
                for i in sentence[1:]:
                    # Targets are padded. So stop appending as soon as a padding or eos token is encountered
                    if i == trg_field.vocab.stoi[trg_field.eos_token] or i == trg_field.vocab.stoi[trg_field.pad_token]:
                        break
                    tmp.append(trg_field.vocab.itos[i])
                _trgs.append([tmp])
            trgs += _trgs
            pred_trg, _ = translate_sentence_vectorized(src, src_field, trg_field, model, device)
            pred_trgs += pred_trg
    return pred_trgs, trgs, bleu_score(pred_trgs, trgs)